In [17]:
import pandas as pd
from transformers import AutoTokenizer, RobertaForTokenClassification
# from roberta import RobertaForTokenClassification
from datasets import load_dataset
from torch import nn
from tqdm import tqdm
import numpy as np
import torch
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

Tag occurences (use this as reference when setting class_weight_beta parameter and ignored_tags list):

[('B-area', 4310), ('B-bookday', 3208), ('B-bookpeople', 3180), ('B-bookstay', 2306), ('B-booktime', 1561), ('B-food', 3804), ('B-name', 1787), ('B-pricerange', 4300), ('B-stars', 1841), ('B-type', 2150), ('I-area', 812), ('I-bookday', 1039), ('I-bookpeople', 226), ('I-bookstay', 2), ('I-booktime', 3120), ('I-food', 2994), ('I-name', 4212), ('I-pricerange', 564), ('I-stars', 1), ('I-type', 1049), ('O', 299338)]

P.S. yes, the dataset really is that bad

In [18]:
TRANSFORMER_MODEL_NAME = 'roberta-base'
epochs = 5
batch_size = 16
learning_rate = 2e-5
class_weight_beta = 0.999999 # 0.99999 (yes, 5 nines) should work ok, increase number of nines if you want stronger imbalance compensation
patience = 1
ignored_tags = ['I-bookstay', 'I-stars']

In [19]:
# Random tests to show how the tokenizer/tokenized function work
tokenizer = AutoTokenizer.from_pretrained(TRANSFORMER_MODEL_NAME)
inputs = tokenizer("We love doing NLI!", padding = 'max_length', return_tensors="pt")
print(tokenizer.decode(inputs.input_ids[0]))
print(inputs.token_to_word(0))
span = inputs.token_to_chars(1)
span[1]
# print(inputs.char_to_token(6))
# inputs

<s>We love doing NLI!</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>

2

In [20]:
def process_intent_list(intent_list):
    intents = set()
    if len(intent_list) == 0:
        intents.add('other')
    for intent in intent_list:
        if intent.startswith('Restaurant'):
            intents.add(intent)
        elif intent.startswith('Hotel'):
            intents.add(intent)
        elif intent.startswith('general'):
            intents.add(intent)
        else:
            intents.add('other')
    # print(f'Original {intent_list}')
    # print(f'Modified {list(intents)}')
    return list(intents)

def preprocess_split(dataset, split):
    df = dataset[split].to_pandas()
    new_df = pd.DataFrame(columns = df.columns)
    for i in range(len(df)):
        # Taken from notebook, to know which lines to skip
        row = df.loc[i]
        if not any(set(row.turns['frames'][turn_id]['service']).intersection(['hotel', 'restaurant']) for turn_id,utt in enumerate(row.turns['utterance'])):
            continue
        
        new_df.loc[len(new_df)] = row
        # new_df.loc[len(new_df) - 1]['services'] = process_service_list(new_df.loc[len(new_df) - 1]['services'])
        # for i, frame_service in [frame['service'] for frame in df.loc[i].turns['frames']]:
            # df.loc[i].turns['frames']
    return new_df

def extract_token_bio_tags(dataset):
    tokens_list = []
    bio_tags_list = []
    useful_pos_list = []
    
    for i in tqdm(range(len(dataset))):
        turns = dataset.loc[i].turns
        for utterance, speaker, dialogue_act, frames in zip(turns['utterance'], turns['speaker'], turns['dialogue_acts'], turns['frames']):
            if speaker != 0:
                continue
            # frames['service'] was used here before
            if 'other' in process_intent_list(dialogue_act['dialog_act']['act_type']):
                continue
            
            span_info = dialogue_act['span_info']
            act_slot_names = span_info['act_slot_name']
            act_slot_values = span_info['act_slot_value']
            span_starts = span_info['span_start']
            span_ends = span_info['span_end']
            slots = {slot_name : {'start': start, 'end': end} for slot_name, start, end in zip(act_slot_names, span_starts, span_ends)}
            
            tokenized = tokenizer(utterance, padding = 'max_length')
            token_tags = [None] * len(tokenized.input_ids)
            
            for j in range(len(token_tags)):
                if tokenized.token_to_word(j) is not None:
                    token_tags[j] = 'O'
            
            for slot_name in slots:
                slot_start, slot_end = slots[slot_name]['start'], slots[slot_name]['end']
                covered_tokens = list(dict.fromkeys(tokenized.char_to_token(k) for k in range(slot_start, slot_end) if utterance[k] != ' '))
                for j, covered_token in enumerate(covered_tokens):
                    bio_type = 'B-' if j == 0 else 'I-'
                    if bio_type + slot_name not in ignored_tags:
                        token_tags[covered_token] = bio_type + slot_name
            
            # print([*zip(tokenizer.convert_ids_to_tokens(tokenized.input_ids), token_tags)])

            tokens_list.append(tokenized)
            bio_tags_list.append(np.array(token_tags))
            useful_pos_list.append((tokenized.char_to_token(0), tokenized.char_to_token(len(utterance) - 1) + 1))
            
    return tokens_list, bio_tags_list, useful_pos_list

In [21]:
dataset = load_dataset('multi_woz_v22')

train = preprocess_split(dataset, 'train')
val = preprocess_split(dataset, 'validation')
test = preprocess_split(dataset, 'test')

train_tokens, train_bio_tags, train_useful_pos = extract_token_bio_tags(train)
possible_bio_tags = sorted(set(filter(lambda tag : tag is not None, np.concatenate(train_bio_tags))))
print(possible_bio_tags)
tag_to_encoding = {tag : encoding for encoding, tag in enumerate(possible_bio_tags)}
# - 100 is default ignore index for the pytorch cross entropy function
tag_to_encoding[None] = -100
encoding_to_tag = {encoding : tag for encoding, tag in enumerate(possible_bio_tags)}
train_encoded_tags = [[tag_to_encoding[tag] for tag in tags] for tags in train_bio_tags]

val_tokens, val_bio_tags, val_useful_pos = extract_token_bio_tags(val)
val_encoded_tags = [[tag_to_encoding[tag] for tag in tags] for tags in val_bio_tags]

test_tokens, test_bio_tags, test_useful_pos = extract_token_bio_tags(test)
test_encoded_tags = [np.array([tag_to_encoding[tag] for tag in tags]) for tags in test_bio_tags]

KeyboardInterrupt: 

Where the idea of the next cell comes from:

https://towardsdatascience.com/handling-class-imbalanced-data-using-a-loss-specifically-made-for-it-6e58fd65ffab

In [ ]:
samples_per_class = [0] * len(possible_bio_tags)
for tags, useful_pos_batch in zip(train_bio_tags, train_useful_pos):
    for tag in tags:
        if tag is not None:
            samples_per_class[tag_to_encoding[tag]] += 1

print("Class counts:")
print([*zip(possible_bio_tags, samples_per_class)])

samples_per_class = np.array(samples_per_class)

effective_num = 1.0 - np.power(class_weight_beta, samples_per_class)
class_weights = (1.0 - class_weight_beta) / effective_num
class_weights = class_weights / np.sum(class_weights) * len(possible_bio_tags)
print("Class weights:")
print([*zip(possible_bio_tags, class_weights)])
class_weights = torch.Tensor(class_weights).cuda()

Class counts:
[('B-area', 4782), ('B-bookday', 4147), ('B-bookpeople', 4015), ('B-bookstay', 2543), ('B-booktime', 2315), ('B-food', 3996), ('B-name', 1951), ('B-pricerange', 4510), ('B-stars', 1913), ('B-type', 2292), ('I-area', 846), ('I-bookday', 1313), ('I-bookpeople', 271), ('I-booktime', 4627), ('I-food', 3150), ('I-name', 4583), ('I-pricerange', 580), ('I-type', 1094), ('O', 394239)]
Class weights:
[('B-area', 0.3185210442402606), ('B-bookday', 0.367177348465117), ('B-bookpeople', 0.3792239199319232), ('B-bookstay', 0.5982951004215206), ('B-booktime', 0.657145172109844), ('B-food', 0.3810234193433338), ('B-name', 0.7796075458359174), ('B-pricerange', 0.3376852898276934), ('B-stars', 0.7950786368108652), ('B-type', 0.6637319308007337), ('I-area', 1.7968964408578951), ('I-bookday', 1.1580573704719834), ('I-bookpeople', 5.607887240224681), ('I-booktime', 0.3291656981047898), ('I-food', 0.4831511288904705), ('I-name', 0.33231861315857447), ('I-pricerange', 2.6206417985146535), ('I-t

RuntimeError: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero.

In [ ]:
def batchify_tokens_tags(tokens_list, encoded_tags_list, useful_pos_list, batch_size):
    ids_batch = []
    mask_batch = []
    useful_pos_batch = []
    labels_batch = []
    
    if encoded_tags_list is None:
        encoded_tags_list = range(len(tokens_list))
    
    if useful_pos_list is None:
        useful_pos_list = range(len(tokens_list))
    
    for tokens, encoded_tags, useful_pos in zip(tokens_list, encoded_tags_list, useful_pos_list):
        ids_batch.append(tokens.input_ids)
        mask_batch.append(tokens.attention_mask)
        useful_pos_batch.append(useful_pos)
        labels_batch.append(encoded_tags)
        
        if len(ids_batch) == batch_size:
            yield torch.Tensor(ids_batch).long().cuda(), torch.Tensor(mask_batch).cuda(), useful_pos_batch, torch.Tensor(labels_batch).long().cuda()
            ids_batch.clear()
            mask_batch.clear()
            useful_pos_batch.clear()
            labels_batch.clear()
    
    yield torch.Tensor(ids_batch).long().cuda(), torch.Tensor(mask_batch).cuda(), useful_pos_batch, torch.Tensor(labels_batch).long().cuda()
    return None

def outputs_keep_useful_part(logits_batch, labels_batch, useful_pos_batch):
    logits_useful = torch.zeros(logits_batch.shape).cuda()
    labels_useful = torch.zeros(labels_batch.shape).cuda().long()
    for i, useful_pos in enumerate(useful_pos_batch):
        logits_useful[i, useful_pos[0] - 1 : useful_pos[1] + 1, :] = logits_batch[i, useful_pos[0] - 1 : useful_pos[1] + 1, :]
        labels_useful[i, useful_pos[0] - 1 : useful_pos[1] + 1] = labels_batch[i, useful_pos[0] - 1 : useful_pos[1] + 1]
    return logits_useful, labels_useful

In [ ]:
def compute_loss(transformer, tokens, encoded_tags, useful_pos_list, batch_size):
    transformer.eval()
    losses = []
    with torch.no_grad():
        for ids_batch, mask_batch, useful_pos_batch, labels_batch in batchify_tokens_tags(tokens, encoded_tags, useful_pos_list, batch_size):
            out = transformer.forward(input_ids = ids_batch, attention_mask = mask_batch)
            
            # logits_useful, labels_useful = outputs_keep_useful_part(out.logits, labels_batch, useful_pos_batch)
            logits_useful = out.logits
            labels_useful = labels_batch
            
            loss = nn.functional.cross_entropy(input = torch.transpose(logits_useful, 1, 2), target = labels_useful, weight = class_weights)
            losses.append(loss.item())
    transformer.train()
    return np.mean(losses)

def predict(transformer, tokens, batch_size):
    transformer.eval()
    predictions = []
    with torch.no_grad():
        for ids_batch, mask_batch, _, _ in batchify_tokens_tags(tokens, None, None, batch_size):
            out = transformer.forward(input_ids = ids_batch, attention_mask = mask_batch, labels = None)
            res = torch.argmax(out.logits, dim = 2).cpu().detach().numpy()
            predictions.append(res)
    return np.concatenate(predictions)

def useful_flattened_tokens(tokens_list, useful_pos_list):
    return np.concatenate([tokens[useful_pos[0] : useful_pos[1]] for tokens, useful_pos in zip(tokens_list, useful_pos_list)])

In [ ]:
transformer = RobertaForTokenClassification.from_pretrained(TRANSFORMER_MODEL_NAME, num_labels = len(possible_bio_tags)).cuda()
transformer.train()
# TODO: maybe use the parameters that the pre-trained model was trained with??
optim = torch.optim.Adam(transformer.parameters(), lr = learning_rate)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForTokenClassification: ['lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able

In [ ]:
print(f"Training for {len(possible_bio_tags)} classes")

train_losses = []
val_losses = []

for epoch in range(epochs):
    epoch_train_loss = []
    
    for ids_batch, mask_batch, useful_pos_batch, labels_batch in batchify_tokens_tags(train_tokens, train_encoded_tags, train_useful_pos, batch_size):
        optim.zero_grad()
        out = transformer.forward(input_ids = ids_batch, attention_mask = mask_batch)
        
        # logits_final, labels_final = outputs_keep_useful_part(out.logits, labels_batch, useful_pos_batch)
        logits_final = out.logits
        labels_final = labels_batch
        
        loss = nn.functional.cross_entropy(input = torch.transpose(logits_final, 1, 2), target = labels_final, weight = class_weights)
        loss.backward()
        optim.step()
        epoch_train_loss.append(loss.item())
    
    epoch_train_loss = np.mean(epoch_train_loss)
    train_losses.append(epoch_train_loss)
    epoch_val_loss = compute_loss(transformer, val_tokens, val_encoded_tags, val_useful_pos, batch_size)
    
    print(f"Epoch {epoch + 1}: Train loss = {epoch_train_loss}, Val loss = {epoch_val_loss}")
    
    if len(val_losses) != 0 and val_losses[-1] <= epoch_val_loss:
        waited += 1
        if waited > patience:
                val_losses.append(epoch_val_loss)
                break
    else:
        waited = 0
    
    val_losses.append(epoch_val_loss)

transformer.save_pretrained('saved_models/SF_' + TRANSFORMER_MODEL_NAME)

plt.plot(train_losses, label = 'Train loss')
plt.plot(val_losses, label = 'Val loss')
plt.legend()
plt.show()

Training for 19 classes


KeyboardInterrupt: 

In [ ]:
with torch.no_grad():
    predicted_encoded_tags = predict(transformer, test_tokens, batch_size)
predicted_encoded_tags_flattened = useful_flattened_tokens(predicted_encoded_tags, test_useful_pos)
test_encoded_tags_flattened = useful_flattened_tokens(test_encoded_tags, test_useful_pos)

report = classification_report(test_encoded_tags_flattened, predicted_encoded_tags_flattened, labels = [*range(len(possible_bio_tags))], target_names = possible_bio_tags, digits = 3, zero_division = 0)
print(report)